In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report, confusion_matrix

In [2]:
!pip install kagglehub

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\shrey\.cache\kagglehub\datasets\masoudnickparvar\brain-tumor-mri-dataset\versions\1


In [3]:
import hashlib


PROJECT_DIR = path
LABELS = ['pituitary','notumor', 'meningioma', 'glioma']
def compute_hash(file):
    hasher = hashlib.md5()
    with open(file, 'rb') as f:
        buf = f.read()
        hasher.update(buf)
    return hasher.hexdigest()


def list_files(hash_dict):
    for data_type in ['Training', 'Testing']:
        for label in LABELS:
            folder_path = os.path.join(PROJECT_DIR, 'data', 'raw', data_type, label)
            for root, dirs, files in os.walk(folder_path):
                for file in files:
                    if file.endswith(".jpg"):
                        file_path = os.path.join(root, file)
                        file_hash = compute_hash(file_path)
                        if file_hash in hash_dict:
                            hash_dict[file_hash].append(file_path)
                        else:
                            hash_dict[file_hash] = [file_path]


def remove_duplicates(hash_dict):
    duplicate_count = 0
    for hash_value, file_paths in hash_dict.items():
        if len(file_paths) > 1:
            for file_path in file_paths[1:]:
                print(f"Removing duplicate (hash : {hash_value}) : {file_path}")
                os.remove(file_path)
                duplicate_count += 1
    print(f"Number of duplicates : {duplicate_count}")


if __name__ == '__main__':
    hash_dict = {}
    list_files(hash_dict)
    remove_duplicates(hash_dict)

Number of duplicates : 0


# Creating data frames of the images:   
This allows me to use the images witht thier filepaths from both the training and testing data set as a data frame instead of uploading each image and plotting it. In my opinion it makes it slighlt simpler.

In [17]:
train_dir = 'C:/Users/shrey/.cache/kagglehub/datasets/masoudnickparvar/brain-tumor-mri-dataset/versions/1/Training'

filepaths = []
labels = []


folds = os.listdir(train_dir)
images=[]
for fold in folds:
    foldpath = os.path.join(train_dir, fold)


    files = os.listdir(foldpath)
    for f in files:
        fpath = os.path.join(foldpath, f)
        #img = Image.open(fpath)
        #images.append(img)

        filepaths.append(fpath)
        labels.append(fold)

#if images:  # Check if the list is not empty
    #plt.imshow(images[0])  # Display the first image
    #plt.axis('off')  # Hide axis for a cleaner look
    #plt.show()
train_df = pd.DataFrame(data={'filepaths':filepaths, 'labels':labels})

'C:/Users/shrey/.cache/kagglehub/datasets/masoudnickparvar/brain-tumor-mri-dataset/versions/1/Training\\meningioma\\Tr-me_0179.jpg'

In [32]:
test_dir = 'C:/Users/shrey/.cache/kagglehub/datasets/masoudnickparvar/brain-tumor-mri-dataset/versions/1/Testing'

filepaths = []
labels = []


folds = os.listdir(test_dir)

for fold in folds:
    foldpath = os.path.join(test_dir, fold)

    files = os.listdir(foldpath)
    for f in files:
        fpath = os.path.join(foldpath, f)
         #img = Image.open(fpath)
        #images.append(img)


        filepaths.append(fpath)
        labels.append(fold)


#if images:  # Check if the list is not empty
    #plt.imshow(images[0])  # Display the first image
    #plt.axis('off')  # Hide axis for a cleaner look
    #plt.show()
ts_df = pd.DataFrame(data={'filepaths':filepaths, 'labels':labels})
filepaths[1250]

'C:/Users/shrey/.cache/kagglehub/datasets/masoudnickparvar/brain-tumor-mri-dataset/versions/1/Testing\\pituitary\\Te-pi_0239.jpg'

In [ ]:
!pip install matplotlib pillow

In [8]:
valid_df, test_df = train_test_split(ts_df, test_size=0.5, random_state=42)

In [9]:
datagen = ImageDataGenerator(rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
gen = ImageDataGenerator(rescale=1.0/255)

train_gen = datagen.flow_from_dataframe(train_df, x_col='filepaths', y_col='labels', target_size=(100, 100),
                                   color_mode='rgb', class_mode='categorical', batch_size=32,subset='training')
valid_gen =  datagen.flow_from_dataframe(valid_df, x_col='filepaths', y_col='labels', target_size=(100,100),
                                     color_mode='rgb', class_mode='categorical', batch_size=32)
test_gen = gen.flow_from_dataframe(test_df, x_col='filepaths', y_col='labels', target_size=(100,100),
                                   color_mode='rgb', class_mode='categorical', batch_size=32,shuffle='False')



Found 5712 validated image filenames belonging to 4 classes.
Found 655 validated image filenames belonging to 4 classes.
Found 656 validated image filenames belonging to 4 classes.


In [10]:
model = Sequential([
    Conv2D(1024, kernel_size=(3, 3), activation='relu', input_shape=(100, 100, 3)),
    MaxPooling2D((2, 2)),

    Conv2D(512, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(256, kernel_size=(3, 3), activation='relu'),
    Conv2D(128, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),

    Dense(256, activation='relu'),
    Dense(128, activation='relu'),

    Dense(4, activation='softmax')
])
model.compile(optimizer=Adamax(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
history = model.fit(train_gen, validation_data=valid_gen, epochs=15)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


179/179 ━━━━━━━━━━━━━━━━━━━━ 114s 466ms/step - accuracy: 0.3720 - loss: 1.2381 - val_accuracy: 0.5389 - val_loss: 1.0586
Epoch 2/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 90s 333ms/step - accuracy: 0.5915 - loss: 0.9466 - val_accuracy: 0.5908 - val_loss: 1.0171
Epoch 3/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 83s 340ms/step - accuracy: 0.6373 - loss: 0.8688 - val_accuracy: 0.6168 - val_loss: 0.8992
Epoch 4/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 80s 324ms/step - accuracy: 0.6914 - loss: 0.7538 - val_accuracy: 0.6489 - val_loss: 0.8650
Epoch 5/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 84s 339ms/step - accuracy: 0.7200 - loss: 0.6965 - val_accuracy: 0.6656 - val_loss: 0.8331
Epoch 6/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 80s 330ms/step - accuracy: 0.7543 - loss: 0.6129 - val_accuracy: 0.7008 - val_loss: 0.6938
Epoch 7/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 82s 326ms/step - accuracy: 0.7709 - loss: 0.5828 - val_accuracy: 0.7084 - val_loss: 0.7106
Epoch 8/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 59s 325ms/step - accuracy: 0.7817 - loss: 0.5662 - va

In [12]:
train_score = model.evaluate(train_gen)
valid_score = model.evaluate(valid_gen)
test_score = model.evaluate(test_gen)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
print("Validation Loss: ", valid_score[0])
print("Validation Accuracy: ", valid_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

179/179 ━━━━━━━━━━━━━━━━━━━━ 31s 175ms/step - accuracy: 0.8745 - loss: 0.3540
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 128ms/step - accuracy: 0.8133 - loss: 0.4593
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - accuracy: 0.7295 - loss: 0.8005
Train Loss:  0.34541624784469604
Train Accuracy:  0.8741246461868286
--------------------
Validation Loss:  0.44634249806404114
Validation Accuracy:  0.8167939186096191
--------------------
Test Loss:  0.7355567216873169
Test Accuracy:  0.7393292784690857
